In [5]:
#准备工作
import cv2,json,os

srcpath='../../input/2020nianhui-img'
dstpath='../../../input/imgsmall/'
imglist=os.listdir(srcpath)
#os.mkdir(dstpath)
'''
for i in imglist:
    path=srcpath+i
    img=cv2.imread(path)
    smallimg=cv2.resize(img, None, fx=0.3, fy=0.3, interpolation=cv2.INTER_AREA)
    cv2.imwrite(dstpath+i,smallimg)
'''
imglist[:3]

['2020-01-06-16-14-24S_542.jpg',
 '2020-01-06-13-06-36S_582.jpg',
 '2020-01-06-16-14-17S_547.jpg']

In [13]:
import base64,numpy as np
import struct
def base64Tofloat(kb_feature):
    # feather = base64.b16decode(kb_feature)
    feature = base64.b64decode(kb_feature)
    float_out = []
    if len(feature) >= 2060:
        for i in range(12,12 + 512 * 4, 4):
            x = feature[i:i + 4]
            float_out.append(struct.unpack('f', x)[0])
    return float_out

In [4]:
import requests
import hashlib
import time
import matplotlib.pyplot as plt
import cv2,json,os
#plt.figure(figsize=(15, 12)) 
url = "http://10.45.154.64:9010/trajectory/reid/feature"
url="http://10.45.154.129/verify/face/detectAndQuality"
#url="http://10.45.154.64:9010/verify/face/detect"

fdetectlog=open('africadetect.log','w')

def detectface_znv(seq,imgname,isShowImg=False,isOutImgWithRect=False):
    files = {'imageData':open(imgname,'rb')}
    reponse = requests.post(url,files=files)

    text = reponse.text
    #print(text)
    text=json.loads(text)
    if text['result']=='error':
        print(text)
        return
    rst=text['data'][0]['result']
    fname=imgname.split('/')[-1][:-4]
    frame=cv2.imread(imgname)
    if rst==None:
        ss=fname+" detect face:none!"
        print(ss)
        fdetectlog.write(ss+'\n')
        return 
    ss=fname+" detect face:%d"%len(rst)+str(rst)
    print(ss)
    fdetectlog.write(ss+'\n')
    cnt=0
    for rec in rst:
        rect=rec['rect']
        #featurestr=rec['feature']
        #if rec['quality']<0.99:
        #    continue
        #feature=base64Tofloat(featurestr)
        #print(feature.shape)
        #print(feature)
        #featurearr.append(feature)
        #print(rect)
        #cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (255, 0, 0), 2)
        rect0=0 if rect[0]-20<0 else rect[0]-20
        rect1=0 if rect[1]-20<0 else rect[1]-20
        rect2=frame.shape[1] if rect[2]+20>frame.shape[1] else rect[2]+20
        rect3=frame.shape[0] if rect[3]+20>frame.shape[0] else rect[3]+20
        
        #cv2.imwrite('africa/%s-%d.jpg'%(fname,cnt),frame[rect[1]:rect[3],rect[0]:rect[2],:])
        cv2.imwrite('africa/%s-face-%d.jpg'%(fname,cnt),frame[rect1:rect3,rect0:rect2,:])
        #imginfo[seq]={"name":i,"score":rec['quality'],"rect":rect}
        cnt=cnt+1

    #截完小图后输出画rect的大图
    if isOutImgWithRect:
        for rec in rst:
            rect=rec['rect']
            cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (255, 0, 0), 2)
    
        cv2.imwrite('africa/%s-rect.jpg'%fname,frame)


    #print (text)
    if isShowImg:
        im2 = frame[:,:,::-1]
        #recx=rst[3]['rect']
        plt.figure(figsize=(40,80))
        plt.imshow(im2)

    
  

In [6]:
import dlib
dlibdetector = dlib.get_frontal_face_detector()
def detectface_dlib(seq,imgname,isShowImg=False,isOutImgWithRect=False):
    fname=imgname.split('/')[-1][:-4]
    frame=cv2.imread(imgname)

    dets = dlibdetector(frame, 1)
    if dets==None:
        ss=fname+" detect face:none!"
        print(ss)
        fdetectlog.write(ss+'\n')
        return 
    ss=fname+" detect face:%d"%len(dets)+str(dets)
    print(ss)
    fdetectlog.write(ss+'\n')
    fdetectlog.flush()
    print("Number of faces detected: {}".format(len(dets)))
    cnt=0
    for i, d in enumerate(dets):
        #print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        #    i, d.left(), d.top(), d.right(), d.bottom()))
        rect0=0 if d.left()-20<0 else d.left()-20
        rect1=0 if d.top()-20<0 else d.top()-20
        rect2=frame.shape[1] if d.right()+20>frame.shape[1] else d.right()+20
        rect3=frame.shape[0] if d.bottom()+20>frame.shape[0] else d.bottom()+20
        
        #cv2.imwrite('africa/%s-%d.jpg'%(fname,cnt),frame[rect[1]:rect[3],rect[0]:rect[2],:])
        cv2.imwrite('africadlib/%s-face-%d.jpg'%(fname,cnt),frame[rect1:rect3,rect0:rect2,:])
        #imginfo[seq]={"name":i,"score":rec['quality'],"rect":rect}
        cnt=cnt+1

    #截完小图后输出画rect的大图
    if isOutImgWithRect:
        for i, d in enumerate(dets):
            cv2.rectangle(frame, (d.left(), d.top()), (d.right(), d.bottom()), (255, 0, 0), 2)
    
        cv2.imwrite('africadlib/%s-rect.jpg'%fname,frame)
    
        #print (text)
    if isShowImg:
        im2 = frame[:,:,::-1]
        #recx=rst[3]['rect']
        plt.figure(figsize=(40,80))
        plt.imshow(im2)

In [7]:
# 正式代码 
imgpath='../../input/africaimages/'
imglist=os.listdir(imgpath)
#imgname='1.jpg'
seq=0
featurearr=[]
imginfo={}
for imgfname in imglist[:1]:
    imgname=imgpath+imgfname
    #imgname='../../../input/imgsmall/2020-01-06-14-01-29S_569.jpg'
    detectface_dlib(seq,imgname)
    #print(imgfname)


11.mp4-000001 detect face:0rectangles[]
Number of faces detected: 0


In [8]:
#单张验证可行性
imgname='../../input/africaimages/11.mp4-000784.jpg'
seq=111
detectface_dlib(seq,imgname,isShowImg=False)
detectface_znv(seq,imgname,isShowImg=False)

11.mp4-000784 detect face:3rectangles[[(194, 727) (237, 770)], [(621, 967) (664, 1010)], [(794, 698) (837, 742)]]
Number of faces detected: 3
{'result': 'error', 'errorMessage': 'SDK_ERROR:_NO_FACE_DETECTED', 'time_used': 2528}


In [29]:
!pip install dlib

     |████████████████████████████████| 3.2MB 6.9kB/s eta 0:00:016
  Created wheel for dlib: filename=dlib-19.19.0-cp36-cp36m-linux_x86_64.whl size=3497780 sha256=8b587b0c7d02ab380ce00f5f0a7c589d09ae9b6189bc4274bfb31e769714ac1e
  Stored in directory: /root/.cache/pip/wheels/96/ac/11/8aadec62cb4fb5b264a9b1b042caf415de9a75f5e165d79a51
Successfully built dlib
You should consider upgrading via the 'pip install --upgrade pip' command.


# 以下是reid测试代码，此处无关

In [95]:
featurearrnp=np.array(featurearr)

In [84]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine

In [97]:

def search(imgfile):
    files = {'imageData':imgfile}

    response = requests.post(url,files=files)

    text = response.text
    rst=json.loads(text)['data'][0]['result']
    if type(rst)!= list and rst.get('errorMessage'):
        return []
    frame=cv2.imread(imgname)
    
    rec=rst[0]
    rect=rec['rect']
    featurestr=rec['feature'][16:]
    import base64,numpy as np
    t=base64.b64decode(featurestr)
    s=str(t,encoding = "utf-8").split(' ')
    feature=np.array(s).astype(np.float32)
    

    cr=1-pairwise_distances([feature],featurearrnp, metric="cosine")
    return cr

In [109]:
imgfile=open('tmp/1.jpg','rb')
cr=search(imgfile)
sortimg = np.argsort(-cr[0]) 
print(sortimg[:30])
ret=[]
for imgidx in sortimg[:16]:
    inf=imginfo[imgidx]
    x={'id':imgidx,'sim':cr[0][imgidx],'name':inf['name'],'score':inf['score'],'rect':inf['rect']}
    print(x)
    ret.append(x)
ret

[217   1 206 191 193  12 202 220 208 205 201 211   4   3  46 190  11  44
  15  49 209   6 219  47   2  14 194 830 855 824]
{'id': 217, 'sim': 0.99714464, 'name': '2020-01-06-16-14-24S_544.jpg', 'score': 0.8836191892623901, 'rect': [524, 275, 668, 716]}
{'id': 1, 'sim': 0.9970808, 'name': '2020-01-06-16-14-24S_542.jpg', 'score': 0.918098509311676, 'rect': [503, 249, 661, 716]}
{'id': 206, 'sim': 0.99675417, 'name': '2020-01-06-16-14-24S_543.jpg', 'score': 0.9299959540367126, 'rect': [500, 258, 669, 744]}
{'id': 191, 'sim': 0.9952736, 'name': '2020-01-06-16-14-17S_545.jpg', 'score': 0.8294156789779663, 'rect': [526, 248, 685, 717]}
{'id': 193, 'sim': 0.99514365, 'name': '2020-01-06-16-14-17S_545.jpg', 'score': 0.6181460618972778, 'rect': [400, 232, 540, 666]}
{'id': 12, 'sim': 0.9950071, 'name': '2020-01-06-16-14-17S_547.jpg', 'score': 0.8688951134681702, 'rect': [520, 246, 682, 722]}
{'id': 202, 'sim': 0.99400175, 'name': '2020-01-06-16-14-17S_546.jpg', 'score': 0.8459798693656921, 'rec

[{'id': 217,
  'sim': 0.99714464,
  'name': '2020-01-06-16-14-24S_544.jpg',
  'score': 0.8836191892623901,
  'rect': [524, 275, 668, 716]},
 {'id': 1,
  'sim': 0.9970808,
  'name': '2020-01-06-16-14-24S_542.jpg',
  'score': 0.918098509311676,
  'rect': [503, 249, 661, 716]},
 {'id': 206,
  'sim': 0.99675417,
  'name': '2020-01-06-16-14-24S_543.jpg',
  'score': 0.9299959540367126,
  'rect': [500, 258, 669, 744]},
 {'id': 191,
  'sim': 0.9952736,
  'name': '2020-01-06-16-14-17S_545.jpg',
  'score': 0.8294156789779663,
  'rect': [526, 248, 685, 717]},
 {'id': 193,
  'sim': 0.99514365,
  'name': '2020-01-06-16-14-17S_545.jpg',
  'score': 0.6181460618972778,
  'rect': [400, 232, 540, 666]},
 {'id': 12,
  'sim': 0.9950071,
  'name': '2020-01-06-16-14-17S_547.jpg',
  'score': 0.8688951134681702,
  'rect': [520, 246, 682, 722]},
 {'id': 202,
  'sim': 0.99400175,
  'name': '2020-01-06-16-14-17S_546.jpg',
  'score': 0.8459798693656921,
  'rect': [520, 248, 677, 729]},
 {'id': 220,
  'sim': 0.992

In [19]:
frame=cv2.imread('africa/11.mp4-000780-0.jpg')
frame.shape[0]

37

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, make_response,jsonify
from werkzeug.utils import secure_filename
import os
import cv2
import time
 
from datetime import timedelta
 
#设置允许的文件格式
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'JPG', 'PNG', 'bmp'])
 
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS
 
app = Flask(__name__)
# 设置静态文件缓存过期时间
app.send_file_max_age_default = timedelta(seconds=1)
 
 
# @app.route('/upload', methods=['POST', 'GET'])
@app.route('/upload', methods=['POST', 'GET'])  # 添加路由
def upload():
    if request.method == 'POST':
        f = request.files['file']
 
        if not (f and allowed_file(f.filename)):
            return jsonify({"error": 1001, "msg": "请检查上传的图片类型，仅限于png、PNG、jpg、JPG、bmp"})
 
        user_input = request.form.get("name")
 
        basepath = os.path.dirname(__file__)  # 当前文件所在路径
 
        upload_path = os.path.join(basepath, 'static/images', secure_filename(f.filename))  # 注意：没有的文件夹一定要先创建，不然会提示没有该路径
        # upload_path = os.path.join(basepath, 'static/images','test.jpg')  #注意：没有的文件夹一定要先创建，不然会提示没有该路径
        f.save(upload_path)
 
        # 使用Opencv转换一下图片格式和名称
        img = cv2.imread(upload_path)
        cv2.imwrite(os.path.join(basepath, 'static/images', 'test.jpg'), img)
 
        return render_template('upload_ok.html',userinput=user_input,val1=time.time())
 
    return render_template('upload.html')
 
 
if __name__ == '__main__':
    # app.debug = True
    app.run(host='0.0.0.0', port=8987, debug=True)


In [94]:
y=np.array(featurearr)
cr=1-pairwise_distances([y[1]],y, metric="cosine")
cr

array([[0.9753269 , 0.99999994, 0.9877226 , ..., 0.9746718 , 0.9585495 ,
        0.9750606 ]], dtype=float32)